# AE353 DP4 Full Code
Written by Joshua Veranga

## 1. Imports

Core libraries and simulation modules used throughout this notebook. Includes:
- `sympy` for symbolic dynamics,
- `numpy` and `scipy` for numerical computations,
- `matplotlib` for plotting,
- `ae353_drone` for drone simulation utilities.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
import pandas as pd
import itertools
import secrets
from matplotlib.patches import Patch
from scipy import linalg

import ae353_drone  # custom module provided for AE353 drone dynamics

# Suppress scientific notation in NumPy output
np.set_printoptions(suppress=True)

## 2. Symbolic Dynamics and Linearization

### 2.1 Parameters

We begin by defining the drone's physical parameters and constructing the symbolic dynamic model. The drone is modeled as a 6-DOF rigid body with net thrust and torques as control inputs.

In [ ]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

### 2.2 Symbolic Equation of Motion and Marker Sensor Model

We define the drone's state variables, compute the symbolic equations of motion $\dot{x} = f(x, u)$, and derive the marker positions used for sensing. The resulting observation model $g(x)$ stacks the left and right rotor marker positions in world coordinates.


In [ ]:
# State variables
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')
psi, theta, phi = sym.symbols('psi, theta, phi')
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])
w_in_body = sym.Matrix([w_x, w_y, w_z])

# Inputs
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')
f_z = sym.symbols('f_z')

# Parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# Rotation matrices and transformation
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi),  sym.cos(psi), 0],
                 [0, 0, 1]])
Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi),  sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# Angular rate mapping
ex, ey, ez = sym.Matrix([[1], [0], [0]]), sym.Matrix([[0], [1], [0]]), sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Force and torque equations
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)
f = sym.simplify(f, full=True)

# Observation model: marker positions
p_in_world = sym.Matrix([p_x, p_y, p_z])
a_in_body = sym.Matrix([0, l, 0])
b_in_body = sym.Matrix([0, -l, 0])
a_in_world = p_in_world + R_body_in_world @ a_in_body
b_in_world = p_in_world + R_body_in_world @ b_in_body
g = sym.simplify(sym.Matrix.vstack(a_in_world, b_in_world))

### 2.3 Linearization and LQR Gain Calculation

We define a helper function to compute the Jacobians of the system and observation model, then solve for the LQR gains $K$ and $L$ using continuous-time Riccati equations.


In [ ]:
def linearize_system(f, g, Qc, Rc, Qo, Ro):
    m = [p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]
    n = [tau_x, tau_y, tau_z, f_z]
    symbols = m + n

    f_num = sym.lambdify(symbols, f)
    g_num = sym.lambdify(symbols, g)

    A_num = sym.lambdify(symbols, f.jacobian(m))
    B_num = sym.lambdify(symbols, f.jacobian(n))
    C_num = sym.lambdify(symbols, g.jacobian(m))
    D_num = sym.lambdify(symbols, g.jacobian(n))

    m_equil = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    n_equil = [0, 0, 0, 9.81 / 2]
    equil = m_equil + n_equil

    A = A_num(*equil)
    B = B_num(*equil)
    C = C_num(*equil)
    D = D_num(*equil)
    marker_equil = g_num(*equil)

    def lqr(A, B, Q, R):
        P = linalg.solve_continuous_are(A, B, Q, R)
        K = linalg.inv(R) @ B.T @ P
        return K


    K = lqr(A, B, Qc, Rc)
    L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T
    
    return A, B, C, K, L, m_equil, n_equil, marker_equil.flatten()

## 3. QWAK Controller Class

This class implements the final LQR-based controller with integrated observer, trajectory generation, and collision avoidance logic. It maintains internal state estimates, computes desired trajectories with valley-based repulsion, and generates control inputs accordingly.


In [ ]:
class QWAK_Controller:
    def __init__(self):
        self.xhat = np.array([0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.dt = 0.04
        self.t = 0
        self.err_max = 2
        self.variables_to_log = ['xhat']

        self.m_equil = np.zeros(12)
        self.n_equil = np.array([0, 0, 0, 4.905])
        self.marker_equil = np.array([0., 0.175, 0., 0., -0.175, 0.])

        self.A = np.array([
            [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 9.81, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, -9.81, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        ])

        self.B = np.array([
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 2],
            [434.7826087, 0, 0, 0],
            [0, 434.7826087, 0, 0],
            [0, 0, 250, 0]
        ])

        self.C = np.array([
            [1, 0, 0, -0.175, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0.175, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0.175, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, -0.175, 0, 0, 0, 0, 0, 0]
        ])

        self.K = np.array([
            [-0, -0.1, -0, -0, -0, 0.48032061, -0, -0.10388681, -0, 0.11049649, -0, -0],
            [0.1, 0, 0, -0, 0.48032061, -0, 0.10388681, 0, 0, -0, 0.11049649, -0],
            [0, 0, -0, 0.1, -0, -0, 0, 0, -0, -0, -0, 0.10392305],
            [-0, 0, 1, -0, -0, -0, -0, 0, 1.04880885, -0, 0, -0]
        ])

        self.L = np.array([
            [2.97537602, 0, 0, 2.97537602, 0, 0],
            [0, 2.92711379, -0.21976816, 0, 2.92711379, 0.21976816],
            [0, 0, 1.09868411, 0, 0, 1.09868411],
            [-2.13087076, 0, 0, 2.13087076, 0, 0],
            [1.30543174, 0, 0, 1.30543174, 0, 0],
            [0, -1.25581808, 0.30429738, 0, -1.25581808, -0.30429738],
            [8.35286245, 0, 0, 8.35286245, 0, 0],
            [0, 8.1162932, -1.19153357, 0, 8.1162932, 1.19153357],
            [0, 0, 0.70710678, 0, 0, 0.70710678],
            [0, -0.67683134, 0.20469329, 0, -0.67683134, -0.20469329],
            [0.70710678, 0, 0, 0.70710678, 0, 0],
            [-0.70710678, 0, 0, 0.70710678, 0, 0]
        ])

    def get_color(self):
        return [1, 1, 1]

    def reset(self, p_x, p_y, p_z, yaw):
        self.xhat = np.array([p_x, p_y, p_z, 0, 0, yaw, 0, 0, 0, 0, 0, 0])
        self.t = 0

    def get_des(self, t, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        pos_hat = xhat[:3]

        if np.linalg.norm(pos_ring - pos_hat) < 1.25:
            x_goal = pos_ring + dir_ring
        else:
            x_goal = pos_ring - dir_ring

        if pos_hat[2] < 1 and not is_last_ring:
            x_goal += np.array([0, 0, 2])

        d = dir_ring / np.linalg.norm(dir_ring)
        delta = pos_hat - pos_ring
        proj = delta - np.dot(delta, d) * d
        dist_to_ring = np.linalg.norm(delta)
        gain = np.clip(dist_to_ring / 3.0, 0.5, 1.5)
        valley_attract = -gain * proj

        repel = np.zeros(3)
        eta = 3.0
        for other in pos_others:
            diff = pos_hat - other
            dist = np.linalg.norm(diff)
            if 0.5 < dist < 4.0:
                repel += eta * diff / (dist ** 3)

        desired_dir = x_goal + valley_attract + repel
        direction = desired_dir - pos_hat
        x_des = pos_hat + self.err_max * direction / np.linalg.norm(direction)

        return np.concatenate([x_des, np.zeros(9)])

    def run(self, pos_markers, pos_ring, dir_ring, is_last_ring, pos_others):
        xdes = self.get_des(self.t, self.xhat, pos_ring, dir_ring, is_last_ring, pos_others)
        u = -self.K @ (self.xhat - xdes)
        y = pos_markers - self.marker_equil
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))
        tau = u + self.n_equil
        self.t += self.dt
        return tau[0], tau[1], tau[2], tau[3]


## 4. Weight Matrix Experiments

### 4.1 LQR Weight Sweep Results

This section plots completion time versus various LQR weight combinations $(\eta, \zeta, \rho)$ to analyze performance sensitivity.

In [ ]:
def run_single_sim(K, L, max_time=70.):
    class TestController(QWAK_Controller):
        def __init__(self):
            super().__init__()
            self.K = K
            self.L = L

    seed = secrets.randbits(32)
    sim = ae353_drone.Simulator(seed=seed, display_pybullet=False)
    sim.add_drone(TestController, 'wark', 'PFP_3.jpg')
    sim.reset()
    sim.run(max_time=max_time, print_debug=False)

    did_fail, did_finish, finish_time = sim.get_result('wark')
    success = int(did_finish and not did_fail)
    return success, finish_time if success else np.nan

# Sweep parameters
Qc_angle_scales = [0.1, 1, 10]    # ψ, θ, ϕ group scaling
Qc_velocity_scales = [0.1, 1, 10] # vx, vy, vz group scaling
Rc_thrust_scales = np.arange(1, 6)  # Rc[3,3] from 1 to 10

# Store results
results = []

# Begin sweep
for q_angle, q_vel, r4 in itertools.product(Qc_angle_scales, Qc_velocity_scales, Rc_thrust_scales):
    # Build Qc and Rc
    Qc = np.diag(
        [1]*3 + [q_angle]*3 + [q_vel]*3 + [1]*3
    )
    Rc = np.diag([100, 100, 100, r4])
    Qo = np.eye(6)
    Ro = np.eye(12)

    A_lin, B_lin, C_lin, K, L, m_equil, n_equil, marker_equil= linearize_system(f, g, Qc, Rc, Qo, Ro)
    success, finish_time = run_single_sim(K, L)

    print(f"Qc=1, angle={q_angle}, vel={q_vel}, AV=1| Rc=100, 100, 100, {r4} | "
            f"Success: {bool(success)} | Time: {finish_time:.2f} s" if success else "Failed")

    results.append({
        'q_angle': q_angle,
        'q_vel': q_vel,
        'r4_thrust': r4,
        'success': success,
        'finish_time': finish_time
    })

### 4.2 LQR Weight Sweep Visualization

This figure displays completion times for different combinations of LQR weights:
- Velocity weight \(\eta \in \{0.1, 1, 10\}\),
- Angle weight \(\zeta \in \{0.1, 1, 10\}\),
- Thrust control weight \(\rho \in \{1, 2, 3, 4, 5\}\).

Each subplot represents a pair of \((\zeta, \eta)\) values, with bars indicating average finish time for each \(\rho\). This allows visual comparison of how weight selection affects performance.


In [ ]:
df_results = pd.DataFrame(results)

# Replace NaN with 0 for failures
df_results['finish_time'] = df_results['finish_time'].fillna(0)

# Unique sorted weights
angle_weights = sorted(df_results['q_angle'].unique())
velocity_weights = sorted(df_results['q_vel'].unique())
rc_values = [1, 2, 3, 4, 5]

# Define color map
cmap = plt.cm.viridis(np.linspace(0, 1, len(rc_values)))

# Set up 3x3 subplots
fig, axes = plt.subplots(3, 3, figsize=(15, 5), sharey=True)
for i, ang in enumerate(reversed(angle_weights)):
    for j, vel in enumerate(velocity_weights):
        ax = axes[i, j]
        
        subset = df_results[(df_results['q_angle'] == ang) & (df_results['q_vel'] == vel)]
        
        heights = []
        for r_val in rc_values:
            avg_time = subset[subset['r4_thrust'] == r_val]['finish_time'].mean()
            heights.append(avg_time)
        
        x = np.arange(len(rc_values))
        bar_width = 0.7
        for idx, (height, color) in enumerate(zip(heights, cmap)):
            ax.bar(x[idx], height, color=color, width=bar_width)

        ax.set_xticks(x)
        ax.set_xticklabels([str(r) for r in rc_values], fontsize=14)
        # ax.set_yticklabels([0,25,50], fontsize=13)
        ax.set_ylim(0, max(df_results['finish_time'].max(), 70))
        
        if i == 2:
            ax.set_xlabel(rf"$\eta = {vel}$", fontsize=15)
        if j == 0:
            ax.set_ylabel(rf"$\zeta = {ang}$", fontsize=15)
        # ax.xticks(fontsize=12)
        # ax.yticks(fontsize=12)
        ax.tick_params(axis='y', labelsize=14)
        ax.grid(axis='y', linestyle='--', alpha=0.6)

# Create a single legend outside the plots
legend_elements = [Patch(facecolor=cmap[i], label=rf"$\rho = {rc_values[i]}$") for i in range(len(rc_values))]
# fig.legend(handles=legend_elements, title=r"$\rho$ (Thrust Control Weight)",
#            loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=5, fontsize=11, title_fontsize=12)
fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=5, fontsize=15, title_fontsize=14)

plt.tight_layout(rect=[0, 0, 1, 0.86])
plt.show()

## 5. Single-Agent Performance Evaluation

### 5.1 Trial Data Collection

This cell runs 100 randomized simulation trials of the `QWAK_Controller` and logs the following performance metrics:

- **Computation time per control step**
- **Finish time for each trial**
- **Success flag (completion without crashing)**
- **Controller position tracking error** (desired vs. actual)
- **Observer estimation error** (estimated vs. actual)

Each trial uses a unique random seed to generate variations in initial conditions and ring heights. Data is collected using the built-in simulator interface and stored for analysis in the subsequent cells.


In [ ]:
# Logging containers
compute_times = []
finish_times = []
success_flags = []
controller_errors = []
observer_errors = []

num_trials = 100  

for _ in range(num_trials):
    seed = secrets.randbits(32)
    print(seed)
    simulator = ae353_drone.Simulator(seed=seed, display_pybullet=False)
    simulator.add_drone(QWAK_Controller, 'qwak', 'PFP_3.jpg')
    simulator.reset()
    simulator.run(max_time=70., print_debug=False)

    did_fail, did_finish, t_finish = simulator.get_result('qwak')
    data = simulator.get_data('qwak')

    # Compute time
    compute_times.extend(data['run_time'])

    # Completion
    success_flags.append(int(did_finish and not did_fail))
    finish_times.append(t_finish if did_finish else np.nan)

    # Position logs
    actual_pos = np.stack([data['p_x'], data['p_y'], data['p_z']], axis=1)
    est_pos = np.array(data['xhat'])[:, :3]

    # Handle desired pos
    if isinstance(data['pos_ring'], list) or np.ndim(data['pos_ring']) == 2:
        desired_array = np.array(data['pos_ring'])  # already a trajectory
    else:
        desired_array = np.tile(data['pos_ring'], (len(actual_pos), 1))  # constant

    # Align lengths
    N = min(len(actual_pos), len(est_pos), len(desired_array))
    actual_pos = actual_pos[:N]
    est_pos = est_pos[:N]
    desired_array = desired_array[:N]

    # Errors
    controller_err = np.linalg.norm(desired_array - actual_pos, axis=1)
    observer_err = np.linalg.norm(est_pos - actual_pos, axis=1)
    controller_errors.extend(controller_err)
    observer_errors.extend(observer_err)

### 5.2 Performance Metrics from 100 Trials

This section visualizes key metrics collected over 100 randomized single-agent trials using the finalized controller.

- Histogram of computation time per control step. A log-scaled y-axis highlights that nearly all steps fall below the 10 ms real-time threshold.
- Histogram of successful flight completion times, centered around 55–58 seconds.
- Histogram of position tracking error $(||x̂ − x^{\text{ref}}||)$ across all time steps.
- Histogram of estimation error $(||x̂ − x||)$, showing the Luenberger observer maintained high accuracy throughout the runs.

All plots are scaled appropriately, with tick sizes enlarged for readability and log-scaling applied to frequency axes where necessary.


In [ ]:
# 1. Computation Time per Step
plt.figure(figsize=(8, 8))
plt.hist(compute_times, bins=200, color='orange', edgecolor='black')
# plt.title("Computation Time per Step", fontsize=16)
plt.xlabel("Time per Step (s)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')
plt.xlim(0,0.010)
plt.grid(True)
plt.show()

# 2. Completion Times
valid_times = [t for t in finish_times if not np.isnan(t)]
plt.figure(figsize=(8, 8))
plt.hist(valid_times, bins=30, color='salmon', edgecolor='black')
# plt.title("Completion Times (Successful Flights Only)", fontsize=16)
plt.xlabel("Completion Time (s)", fontsize=20)
plt.ylabel("Number of Flights", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.grid(True)
plt.show()

# 3. Controller Tracking Error
plt.figure(figsize=(8, 8))
plt.hist(controller_errors, bins=30, color='skyblue', edgecolor='black')
# plt.title("Controller Tracking Error", fontsize=16)
plt.xlabel("Position Error (m)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')
plt.grid(True)
plt.show()

# 4. Observer Estimation Error
plt.figure(figsize=(8, 8))
plt.hist(observer_errors, bins=30, color='lightgreen', edgecolor='black')
# plt.title("Observer Estimation Error", fontsize=16)
plt.xlabel("Estimation Error (m)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.yscale('log')
plt.grid(True)
plt.show()

## 6. Multi-Agent Collision Avoidance Evaluation

### 6.1 Definition of Controller Variants

To evaluate the effect of gain tuning on collision avoidance robustness, we define two alternate LQR-Observer controller configurations with distinct weighting matrices:

Each controller is initialized using the `linearize_system` method, which returns the linearized system matrices and gain values for the respective configurations. The observer weights \( Q_o \), \( R_o \) remain identity matrices.


In [ ]:
Qc1 = np.diag([1, 1, 1, .1, .1, .1, .1, .1, .1, 1, 1, 1])
Rc1 = np.diag([100, 100, 100, 5])

Qo1 = np.diag([1, 1, 1, 1, 1, 1])
Ro1 = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

A1, B1, C1, K1, L1, m_equil1, n_equil1, marker_equil1 = linearize_system(f, g, Qc1, Rc1, Qo1, Ro1)

Qc2 = np.diag([1, 1, 1, 10, 10, 10, .1, .1, .1, 1, 1, 1])
Rc2 = np.diag([100, 100, 100, 4])

Qo2 = np.diag([1, 1, 1, 1, 1, 1])
Ro2 = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

A2, B2, C2, K2, L2, m_equil2, n_equil2, marker_equil2 = linearize_system(f, g, Qc2, Rc2, Qo2, Ro2)

### 6.2 Alternate Controller Implementations: WARK and KUPO

To compare the effects of different controller gain configurations on collision avoidance performance, two additional controllers were implemented:

- **WARK Controller** uses the gain matrices from a controller which emphasize smoother, less aggressive responses.
- **KUPO Controller** uses gain matrices from a controller which apply stronger velocity penalties and reduced attitude weighting.

Both controllers share the same structural design as the primary `QWAK_Controller`, including the LQR-based feedback law, Luenberger observer, and repulsion-based collision avoidance in the `get_des` method. Differences in behavior arise solely from the differing weight matrices and resulting gains.


In [ ]:
class WARK_Controller:
    def __init__(self):
        self.xhat = np.array([0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.dt = 0.04
        self.t = 0
        self.err_max = 2
        self.variables_to_log = ['xhat']

        self.A = A1
        self.B = B1
        self.C = C1
        self.K = K1
        self.L = L1
        self.m_equil = m_equil1
        self.n_equil = n_equil1
        self.marker_equil = marker_equil1

    def get_color(self):
        return [148, 0, 211]

    def reset(self, p_x, p_y, p_z, yaw):
        self.xhat = np.array([p_x, p_y, p_z, 0, 0, yaw, 0, 0, 0, 0, 0, 0])
        self.t = 0

    def get_des(self, t, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        pos_hat = xhat[:3]

        if np.linalg.norm(pos_ring - pos_hat) < 1.25:
            x_goal = pos_ring + dir_ring
        else:
            x_goal = pos_ring - dir_ring

        if pos_hat[2] < 1 and not is_last_ring:
            x_goal += np.array([0, 0, 2])

        d = dir_ring / np.linalg.norm(dir_ring)
        delta = pos_hat - pos_ring
        proj = delta - np.dot(delta, d) * d
        dist_to_ring = np.linalg.norm(delta)
        gain = np.clip(dist_to_ring / 3.0, 0.5, 1.5)
        valley_attract = -gain * proj

        repel = np.zeros(3)
        eta = 3.0
        for other in pos_others:
            diff = pos_hat - other
            dist = np.linalg.norm(diff)
            if 0.5 < dist < 4.0:
                repel += eta * diff / (dist ** 3)

        desired_dir = x_goal + valley_attract + repel
        direction = desired_dir - pos_hat
        x_des = pos_hat + self.err_max * direction / np.linalg.norm(direction)

        return np.concatenate([x_des, np.zeros(9)])

    def run(self, pos_markers, pos_ring, dir_ring, is_last_ring, pos_others):
        xdes = self.get_des(self.t, self.xhat, pos_ring, dir_ring, is_last_ring, pos_others)
        u = -self.K @ (self.xhat - xdes)
        y = pos_markers - self.marker_equil
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))
        tau = u + self.n_equil
        self.t += self.dt
        return tau[0], tau[1], tau[2], tau[3]


In [ ]:
class KUPO_Controller:
    def __init__(self):
        self.xhat = np.array([0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.dt = 0.04
        self.t = 0
        self.err_max = 2
        self.variables_to_log = ['xhat']

        self.A = A2
        self.B = B2
        self.C = C2
        self.K = K2
        self.L = L2
        self.m_equil = m_equil2
        self.n_equil = n_equil2
        self.marker_equil = marker_equil2

    def get_color(self):
        return [148, 0, 211]

    def reset(self, p_x, p_y, p_z, yaw):
        self.xhat = np.array([p_x, p_y, p_z, 0, 0, yaw, 0, 0, 0, 0, 0, 0])
        self.t = 0

    def get_des(self, t, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        pos_hat = xhat[:3]

        if np.linalg.norm(pos_ring - pos_hat) < 1.25:
            x_goal = pos_ring + dir_ring
        else:
            x_goal = pos_ring - dir_ring

        if pos_hat[2] < 1 and not is_last_ring:
            x_goal += np.array([0, 0, 2])

        d = dir_ring / np.linalg.norm(dir_ring)
        delta = pos_hat - pos_ring
        proj = delta - np.dot(delta, d) * d
        dist_to_ring = np.linalg.norm(delta)
        gain = np.clip(dist_to_ring / 3.0, 0.5, 1.5)
        valley_attract = -gain * proj

        repel = np.zeros(3)
        eta = 3.0
        for other in pos_others:
            diff = pos_hat - other
            dist = np.linalg.norm(diff)
            if 0.5 < dist < 4.0:
                repel += eta * diff / (dist ** 3)

        desired_dir = x_goal + valley_attract + repel
        direction = desired_dir - pos_hat
        x_des = pos_hat + self.err_max * direction / np.linalg.norm(direction)

        return np.concatenate([x_des, np.zeros(9)])

    def run(self, pos_markers, pos_ring, dir_ring, is_last_ring, pos_others):
        xdes = self.get_des(self.t, self.xhat, pos_ring, dir_ring, is_last_ring, pos_others)
        u = -self.K @ (self.xhat - xdes)
        y = pos_markers - self.marker_equil
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))
        tau = u + self.n_equil
        self.t += self.dt
        return tau[0], tau[1], tau[2], tau[3]


### 6.3 Multi-Agent Race Evaluation

To evaluate the controllers under competitive multi-agent conditions, we conducted 100 randomized race trials where three drones—Qwak, Wark, and Kupo—flew simultaneously using different controller gains.

In each trial:
- A unique seed was used to randomize ring heights and initial conditions.
- All three drones were spawned and raced through the same course.
- Each drone's result was recorded, including whether it finished and its completion time.

After all trials, we computed the total number of finishes and the average finish time for each drone. The number of drones completing the course in each trial was also tracked to assess overall completion robustness in shared environments.


In [ ]:
num_trials = 100

drone_names = ['qwak', 'wark', 'kupo']
indiv_count = [0, 0, 0]
total_finish_times = [[], [], []]
num_finished_drones = []

for trial in range(num_trials):
    seed = secrets.randbits(32)
    print(f"\nTrial {trial + 1}, Seed: {seed}")

    sim = ae353_drone.Simulator(seed=seed, display_pybullet=False)
    sim.add_drone(QWAK_Controller, 'qwak', 'PFP_3.jpg')
    sim.add_drone(WARK_Controller, 'wark', 'PFP_3.jpg')
    sim.add_drone(KUPO_Controller, 'kupo', 'PFP_3.jpg')
    
    sim.reset()
    sim.run(max_time=70., print_debug=False)
    
    finished_count = 0

    print("Drones that finished:")

    for name in drone_names:
        did_fail, did_finish, t_finish = sim.get_result(name)
        if did_finish and not did_fail:
            print(f' - {name} : {t_finish:.2f} seconds')
            indiv_count[drone_names.index(name)] += 1

            total_finish_times[drone_names.index(name)].append(t_finish)

            finished_count += 1
    num_finished_drones.append(finished_count)

print("\n====== Summary ======")
print(f"Average Number of Drones Finished: {np.mean(num_finished_drones):.2f} / {3}")
print(f" - Qwak finished {indiv_count[0]} times with an average finish time of {np.mean(total_finish_times[0]):.2f} seconds")
print(f" - Wark finished {indiv_count[1]} times with an average finish time of {np.mean(total_finish_times[1]):.2f} seconds")
print(f" - Kupo finished {indiv_count[2]} times with an average finish time of {np.mean(total_finish_times[2]):.2f} seconds")
